In [ ]:
import sys
import os
import pandas as pd, numpy as np, seaborn as sns
import matplotlib.pyplot as plt
import math
from scipy import stats
from scipy.stats import poisson
from sklearn.preprocessing import LabelEncoder
import pylab
from scipy.stats import norm

In [ ]:
biosamples = 733
num_components = 16
num_dhs = 3591898
calls = pd.read_csv('../../ranking_metrics/results/calls',sep='\t',header=None)
vocab = pd.read_csv('../../dhs_data/DHS_Index_and_Vocabulary_hg38_WM20190703.txt',sep='\t')
regions = pd.read_csv('../data/new_maxMean.bed',sep='\t',header=None)
clr_scheme = {'Placental / trophoblast': '#FFE500', 'Lymphoid': '#FE8102', 'Myeloid / erythroid': '#FF0000',
              'Cardiac': '#07AF00','Musculoskeletal': '#4C7D14', 'Vascular / endothelial': '#414613',
              'Primitive / embryonic': '#05C1D9', 'Neural': '#0467FD', 'Digestive': '#009588',
              'Stromal A': '#BB2DD4', 'Stromal B': '#7A00FF', 'Renal / cancer': '#4A6876', 
              'Cancer / epithelial': '#08245B', 'Pulmonary devel.': '#B9461D','Organ devel. / renal': '#692108',
              'Tissue invariant': '#C3C3C3'}
chrs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']
snr=pd.read_csv('../../ranking_metrics/results/sorted_zscored_peak_snr',sep='\t')
mcs=pd.read_csv('../../ranking_metrics/results/sorted_zscored_mean_cosine_similarity',sep='\t')
metric = pd.read_csv('../../ranking_metrics/results/coranked_metrics3',sep='\t')

In [ ]:
def gather_all_anno(ch,snr,mcs):
    ch='chr'+str(ch)
    chr_coranks=pd.read_csv('../results/corrected_coranking/'+ch,sep='\t')
    unbiased_metric_contribution=pd.DataFrame(columns=['mean_snr_ranks','mean_mcs_ranks'])
    for window in chr_coranks.index:
        dhs_indices_in_window=np.where((vocab.seqname==ch) & (vocab.start >= chr_coranks.iloc[window,1]) & (vocab.start <= chr_coranks.iloc[window,2]) )[0]
        unbiased_metric_contribution.loc[window,'mean_mcs_ranks']=np.nanmean(mcs.index[mcs.dhs_index.isin(dhs_indices_in_window)])
        unbiased_metric_contribution.loc[window,'mean_snr_ranks']=np.nanmean(snr.index[snr.dhs_index.isin(dhs_indices_in_window)])
    all_annotations=pd.concat([chr_coranks,unbiased_metric_contribution],axis=1)
    all_annotations.to_csv('../results/corrected_all_anno/'+ch,sep='\t',index=False)

In [ ]:
for i in chrs:
    gather_all_anno(i,snr,mcs)

In [ ]:
all_chr_enrich=pd.DataFrame()
for i in chrs:
    ch='chr'+str(i)
    a=pd.read_csv('../results/component_enrichment_new_maxMean/'+ch+'_comp_enrich_coranks3_new_maxMean',sep='\t')
    all_chr_enrich=pd.concat([all_chr_enrich,a],axis=0)
all_chr_enrich=all_chr_enrich.drop("Unnamed: 19",axis=1).reset_index(drop=True)
all_chr_enrich

In [ ]:
def corank_ch(m):
    ch = 'chr'+str(m)
    chr_ce=pd.read_csv('../results/corrected_ce_new_maxMean/'+ch+'_ce',sep='\t')
    chr_info = pd.read_csv('../results/summary_stats_new_maxMean/'+ch+'_stats_coranks3_new_maxMean',sep='\t')
    chr_ce=chr_ce.drop('Unnamed: 19',axis=1)
    sorted_ce=chr_ce.iloc[chr_ce.iloc[:,3:].max(axis=1).sort_values(ascending=False).index,:].reset_index()
    z = (chr_info.mean_coranked_metrics3-population_mean)/(population_std_dev/np.sqrt(chr_info.no_of_dhs))
    pvals_chr=pd.DataFrame(columns=['pvalue'])
    for j in range(len(z)):
        if z[j]>0:
            pvals_chr.loc[j,'pvalue'] = 1-norm.cdf(z[j])
        else:
            pvals_chr.loc[j,'pvalue'] = norm.cdf(z[j])

    #pvals_chr=pd.DataFrame(data=norm.sf(abs(chr_info.mean_coranked_metrics3-population_mean)/(population_std_dev/np.sqrt(chr_info.no_of_dhs)))*2,columns=['pvalue'])
    pvals_chr=pvals_chr.sort_values(by='pvalue')
    sorted_quality=pd.concat([chr_info.iloc[pvals_chr.index,:],pvals_chr],axis=1)
    sorted_quality=sorted_quality.reset_index()
    chr_corank_quality_density_component_distribution = chr_info[['seqname','start','end','no_of_dhs']].copy()
    for i in chr_info.index:
        rank1 = np.where(sorted_quality.iloc[:,0]==i)[0][0]
        rank2 = np.where(sorted_ce.iloc[:,0]==i)[0][0]
        chr_corank_quality_density_component_distribution.loc[i,'mean_corank']=(rank1+rank2)/2
        chr_corank_quality_density_component_distribution.loc[i,'clt_pvalue']=pvals_chr.loc[i,'pvalue']
        chr_corank_quality_density_component_distribution.loc[i,'max_enrich']=max(chr_ce.iloc[i,3:])
        if rank1 >= rank2:
            chr_corank_quality_density_component_distribution.loc[i,'max_metric']='clt_pvalue'
        else:
            chr_corank_quality_density_component_distribution.loc[i,'max_metric']='component_enrichment'
    chr_corank_quality_density_component_distribution.sort_values(by='mean_corank').reset_index(drop=True).to_csv('../results/corrected_coranking/'+ch,sep='\t',index=False)
    #chr_corank_quality_density_component_distribution.sort_values(by='mean_corank').reset_index()

In [ ]:
for i in chrs:
    corank_ch(i)

## Global Co-ranking

In [ ]:
all_chr=pd.DataFrame()
for i in chrs:
    ch='chr'+str(i)
    a=pd.read_csv('../results/corrected_all_anno/'+ch,sep='\t')
    all_chr=pd.concat([all_chr,a],axis=0)
all_chr=all_chr.reset_index(drop=True)
sorted_clt=all_chr.sort_values(by='clt_pvalue').reset_index()
sorted_enrich=all_chr.sort_values(by='max_enrich',ascending=False).reset_index()
for i in all_chr.index:
    rank1=np.where(sorted_clt.iloc[:,0]==i)[0][0]
    rank2=np.where(sorted_enrich.iloc[:,0]==i)[0][0]
    all_chr.loc[i,'global_mean_coranks']=(rank1+rank2)/2
    all_chr.loc[i,'global_CLT_rank'] = rank1
    all_chr.loc[i,'global_enrichment_rank'] = rank2

In [ ]:
def winner_component():
    #all_chr_enrich=pd.DataFrame()
    for i in chrs:
        ch='chr'+str(i)
        a=pd.read_csv('../results/component_enrichment_new_maxMean/'+ch+'_comp_enrich_coranks3_new_maxMean',sep='\t')
        #all_chr_enrich=pd.concat([all_chr_enrich,a],axis=0)
        #all_chr_enrich=all_chr_enrich.drop("Unnamed: 19",axis=1).reset_index(drop=True)
        a=a.drop("Unnamed: 19",axis=1)
        for j in a.index:
            print(a.columns[np.where(a.iloc[j,3:] == np.max(a.iloc[j,3:]))[0][0]+3])


In [ ]:
all_chr_enrich=pd.DataFrame()
for i in chrs:
    ch='chr'+str(i)
    a=pd.read_csv('../results/corrected_ce_new_maxMean/'+ch+'_ce',sep='\t')
    all_chr_enrich=pd.concat([all_chr_enrich,a],axis=0)
all_chr_enrich=all_chr_enrich.drop("Unnamed: 19",axis=1).reset_index(drop=True)
winner_comp=pd.read_csv('../results/win_comp_all',sep='\t')
x=pd.concat([all_chr_enrich.iloc[:,0:3],winner_comp],axis=1)
all_chr=all_chr.merge(x,left_on=['seqname','start','end'],right_on=['seqname','start','end'])
all_chr=all_chr.sort_values('global_mean_coranks').reset_index(drop=True)
all_chr.to_csv('../results/corrected_global_coranking',sep='\t',index=False)